In [1]:
import os
import numpy as np
import nltk
from keras.models import Sequential
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, recurrent, Embedding,Dropout, Bidirectional
from keras.preprocessing.sequence import pad_sequences
import keras
import json
from data_util import *

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def create_model(x_vocab_len, x_max_len, y_vocab_len, y_max_len, hidden_size, num_layers):
    model = Sequential()

    # Creating encoder network
    model.add(Embedding(x_vocab_len, 1024, input_length=x_max_len, mask_zero=True))
    model.add(Bidirectional(GRU(hidden_size)))
    model.add(Dropout(0.2))
    model.add(RepeatVector(y_max_len))

    # Creating decoder network
    for _ in range(num_layers):
        model.add(GRU(hidden_size * 2, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(TimeDistributed(Dense(y_vocab_len)))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
            optimizer='rmsprop',
            metrics=['accuracy'])
    return model

In [3]:
SENTENCE_LENGTH = 20
VOCAB_SIZE = 10000


In [4]:
def load_cornell_movie_data(start_line, limit, vocab_dict):
    sentences = read_lines("movie_lines.txt", " +++$+++ ", 3, 4, start_line, limit)
    x_vec, y_vec = vectorize_sentence(sentences, vocab_dict, VOCAB_SIZE, SENTENCE_LENGTH)
    return x_vec, y_vec

def load_twitter_data(start_line, limit, vocab_dict):
    sentences = read_sentences("ShortenTwitterAsciiCorpus.txt")[1000:2000]
    x_vec, y_vec = vectorize_sentence(sentences, vocab_dict, VOCAB_SIZE, SENTENCE_LENGTH)
    return x_vec, y_vec

In [47]:
model = create_model(VOCAB_SIZE, SENTENCE_LENGTH, VOCAB_SIZE, SENTENCE_LENGTH, 100, 3)
model.summary()
model.load_weights("0new_model.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 1024)          10240000  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               675000    
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 20, 200)           0         
_________________________________________________________________
gru_6 (GRU)                  (None, 20, 200)           240600    
_________________________________________________________________
gru_7 (GRU)                  (None, 20, 200)           240600    
_________________________________________________________________
gru_8 (GRU)                  (None, 20, 200)           240600    
__________

In [6]:
vocab_dict, idx_dict = load_vocab()

In [ ]:
batch_size = 1000
start_point = 77000
batch = 77
round = 0
while True:
    print("Getting data:", start_point, batch_size)
    x_vec, y_vec = load_cornell_movie_data(start_point, batch_size, vocab_dict)
    if len(x_vec) == 0:
        start_point = 0
        batch = 0
        x_vec, y_vec = load_cornell_movie_data(start_point, batch_size, vocab_dict)
        round += 1 
    model.fit(x_vec, y_vec, batch_size=100, nb_epoch=1500, shuffle = True)
    model.save_weights(str(batch%2)+"new_model.h5")
    start_point += batch_size
    batch += 1
    print("--------------------------------------------------------------------------")
    print("Round:", round, "Batch:", batch)
    print("--------------------------------------------------------------------------")

Getting data: 77000 1000
(452, 20, 10000)


/usr/local/lib/python3.5/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1500
452/452 [==============================] - 5s 11ms/step - loss: 5.0748 - acc: 0.4184
Epoch 2/1500
452/452 [==============================] - 2s 4ms/step - loss: 4.4820 - acc: 0.4583
Epoch 3/1500
452/452 [==============================] - 2s 4ms/step - loss: 4.2640 - acc: 0.4723
Epoch 4/1500
452/452 [==============================] - 2s 4ms/step - loss: 4.0989 - acc: 0.4841
Epoch 5/1500
452/452 [==============================] - 2s 4ms/step - loss: 4.0264 - acc: 0.4929
Epoch 6/1500
452/452 [==============================] - 2s 4ms/step - loss: 3.9171 - acc: 0.4950
Epoch 7/1500
452/452 [==============================] - 2s 4ms/step - loss: 3.8691 - acc: 0.5013
Epoch 8/1500
452/452 [==============================] - 2s 4ms/step - loss: 3.7851 - acc: 0.5054
Epoch 9/1500
452/452 [==============================] - 2s 4ms/step - loss: 3.7064 - acc: 0.5067
Epoch 10/1500
452/452 [==============================] - 2s 4ms/step - loss: 3.6622 - acc: 0.5127
Epoch 11/1500
452/452 [=====

452/452 [==============================] - 2s 4ms/step - loss: 2.4364 - acc: 0.6424
Epoch 85/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.4254 - acc: 0.6394
Epoch 86/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.4172 - acc: 0.6427
Epoch 87/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.3985 - acc: 0.6458
Epoch 88/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.3823 - acc: 0.6520
Epoch 89/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.3929 - acc: 0.6467
Epoch 90/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.3711 - acc: 0.6531
Epoch 91/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.3801 - acc: 0.6478
Epoch 92/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.3954 - acc: 0.6450
Epoch 93/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.3778 - acc: 0.6476
Epoch 94/1500
452/452 [===========

452/452 [==============================] - 2s 4ms/step - loss: 2.0670 - acc: 0.7022
Epoch 168/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.0701 - acc: 0.6986
Epoch 169/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.0675 - acc: 0.7002
Epoch 170/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.0655 - acc: 0.7003
Epoch 171/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.0564 - acc: 0.6990
Epoch 172/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.0647 - acc: 0.7032
Epoch 173/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.0441 - acc: 0.7046
Epoch 174/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.0720 - acc: 0.6962
Epoch 175/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.0603 - acc: 0.7035
Epoch 176/1500
452/452 [==============================] - 2s 4ms/step - loss: 2.0464 - acc: 0.7019
Epoch 177/1500
452/452 [=

...

456/456 [==============================] - 2s 4ms/step - loss: 1.7287 - acc: 0.7451
Epoch 365/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.7509 - acc: 0.7401
Epoch 366/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.7204 - acc: 0.7442
Epoch 367/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.7253 - acc: 0.7446
Epoch 368/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.7291 - acc: 0.7406
Epoch 369/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.7323 - acc: 0.7463
Epoch 370/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.7756 - acc: 0.7346
Epoch 371/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.7254 - acc: 0.7461
Epoch 372/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.7293 - acc: 0.7439
Epoch 373/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.7197 - acc: 0.7500
Epoch 374/1500
456/456 [=

456/456 [==============================] - 2s 4ms/step - loss: 1.6006 - acc: 0.7662
Epoch 447/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.6133 - acc: 0.7641
Epoch 448/1500
456/456 [==============================] - 2s 5ms/step - loss: 1.6055 - acc: 0.7661
Epoch 449/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.6269 - acc: 0.7614
Epoch 450/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.5985 - acc: 0.7689
Epoch 451/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.5913 - acc: 0.7677
Epoch 452/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.5983 - acc: 0.7649
Epoch 453/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.6070 - acc: 0.7671
Epoch 454/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.6056 - acc: 0.7702
Epoch 455/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.5849 - acc: 0.7696
Epoch 456/1500
456/456 [=

456/456 [==============================] - 2s 4ms/step - loss: 1.5048 - acc: 0.7872
Epoch 529/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.5097 - acc: 0.7884
Epoch 530/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.5128 - acc: 0.7875
Epoch 531/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.5281 - acc: 0.7821
Epoch 532/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.5117 - acc: 0.7844
Epoch 533/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.5264 - acc: 0.7806
Epoch 534/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.5054 - acc: 0.7893
Epoch 535/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.5128 - acc: 0.7831
Epoch 536/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.5029 - acc: 0.7851
Epoch 537/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.5199 - acc: 0.7841
Epoch 538/1500
456/456 [=

456/456 [==============================] - 2s 4ms/step - loss: 1.4273 - acc: 0.8064
Epoch 611/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.4474 - acc: 0.7986
Epoch 612/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.4266 - acc: 0.8043
Epoch 613/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.4409 - acc: 0.7985
Epoch 614/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.4393 - acc: 0.8015
Epoch 615/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.4339 - acc: 0.7991
Epoch 616/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.4394 - acc: 0.8019
Epoch 617/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.4331 - acc: 0.8011
Epoch 618/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.4359 - acc: 0.8022
Epoch 619/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.4402 - acc: 0.7958
Epoch 620/1500
456/456 [=

456/456 [==============================] - 2s 4ms/step - loss: 1.3670 - acc: 0.8181
Epoch 693/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.3782 - acc: 0.8156
Epoch 694/1500
456/456 [==============================] - 2s 4ms/step - loss: 1.3792 - acc: 0.8162
Epoch 695/1500
400/456 [=========================>....] - ETA: 0s - loss: 1.4083 - acc: 0.8054

In [50]:
sen = "hey"
sen = sen.lower().replace('?', '').replace('!', '').replace( '.', '').split(" ")
vec = sentence_to_vec([sen], vocab_dict, 'UNK', SENTENCE_LENGTH)
#print(model.predict(vec).shape)
res = model.predict(vec)

vec_y = np.argmax(res, axis=2)
" ".join([idx_dict[x] for x in vec_y[0]])

"              UNK               he's   i is anything"

In [ ]:
#do you clean
sen = "what the hell"
sen = sen.lower().replace('?', '').replace('!', '').replace( '.', '').split(" ")
vec = sentence_to_vec([sen], vocab_dict, 'UNK', SENTENCE_LENGTH)
#print(model.predict(vec).shape)
res = model.predict(vec)

vec_y = np.argmax(res, axis=2)
" ".join([idx_dict[x] for x in vec_y[0] if idx_dict[x]!= 'UNK'])